# Chapter 9
## 1. 설치
### 관리자 귄한으로 cmd, 또는 anaconda prompt 실행 후 다음 명령어를 입력
- pip install --upgrade setuptools
- pip install tensorflow
- pip install keras
- pip install sklearn

## 2. 첫번째 계산 그래프를 만들어 세션에서 실행하기

In [1]:
import tensorflow as tf

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x**2*y + y + 2 # 계산 그래프 생성

In [2]:
# 세션을 만들고 변수를 초기화한 다음 함수 f를 평가하고 세션을 닫음

sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


In [3]:
# 더 간단한 방법

with tf.Session() as sess:
    x.initializer.run() # tf.get_default_session().run(x.initializer) 를 호출하는 것과 동일
    y.initializer.run()
    result = f.eval() # tf.get_default_session().run(f) 를 호출하는 것과 동일 

In [4]:
init = tf.global_variables_initializer() # 초기화를 바로 수행하지 않고 계산 그래프가 실행될 때 모든 변수를 초기화할 노드 생성

with tf.Session() as sess:
    init.run() # 실제 모든 변수 초기화
    result = f.eval()

In [5]:
sess = tf.InteractiveSession() # 자동으로 자신을 기본 세션으로 지정
init.run()
result = f.eval()
print(result)
sess.close()

42


#### tensorflow program의 구성
- 계산 그래프 만들기 (구성단계)
   - 훈련에 필요한 계산과 머신러닝 모델을 표현한 계산 그래프 만들기 
- 그래프 실행 (실행단계)
   - 훈련 스텝을 반복해서 평가하고, 모델 파라미터를 점진적으로 개선하기 위해 반복 루프 수행

## 3. 계산 그래프 관리

In [6]:
# 노드를 만들면 자동으로 기본 계산 그래프에 추가됨
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [7]:
# 독립적인 여러 개의 계산 그래프 만들기
graph = tf.Graph()
with graph.as_default():
    x1 = tf.Variable(1)
    x2 = tf.Variable(2)

In [8]:
x2.graph is graph

True

In [9]:
x2.graph is tf.get_default_graph()

False

In [10]:
# tf.reset_default_graph() 로 기본 그래프를 초기화할 수 있음 
# 실험적인 작업을 하는 동안 기본 그래프에 중복된 노드가 많이 포함됨

## 4. 노드 값의 생애주기

In [11]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3
# x는 w에 의존, y와 z는 x에 의존함을 자동으로 감지

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


In [12]:
# y와 z를 한번에 평가
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


## 5. tensorflow를 이용한 linear regression

### 캘리포니아 주택 가격 데이터셋에 선형 회귀를 수행

In [14]:
# 데이터셋 추출
import numpy as np
from sklearn.datasets import fetch_california_housing

# 모든 훈련 샘플에 편향에 대한 특성(x0=1) 입력
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

# 두개의 텐서플로 상수 노드 X(data), y(target)를 만들기
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)),XT), y) # normal equation

with tf.Session() as sess:
    theta_value = theta.eval()

In [15]:
theta_value

array([[-3.6894890e+01],
       [ 4.3661433e-01],
       [ 9.4453208e-03],
       [-1.0704148e-01],
       [ 6.4345831e-01],
       [-3.9632569e-06],
       [-3.7880042e-03],
       [-4.2093179e-01],
       [-4.3400639e-01]], dtype=float32)

## 6. 경사하강법(gradient descent) 구현

### 직접 gradient 계산

In [16]:
# 특성 벡터의 스케일 조정 (훈련 속도 절감)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [17]:
print(scaled_housing_data_plus_bias.mean(axis=0))
print(scaled_housing_data_plus_bias.mean(axis=1))
print(scaled_housing_data_plus_bias.mean())
print(scaled_housing_data_plus_bias.shape)

[ 1.00000000e+00  6.60969987e-17  5.50808322e-18  6.60969987e-17
 -1.06030602e-16 -1.10161664e-17  3.44255201e-18 -1.07958431e-15
 -8.52651283e-15]
[ 0.38915536  0.36424355  0.5116157  ... -0.06612179 -0.06360587
  0.01359031]
0.11111111111111005
(20640, 9)


In [27]:
n_epochs = 1000 # 훈련단계 1000번 반복
learning_rate = 0.01 # 학습률 eta 설정

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse") # mean squared error
gradients = 2/m * tf.matmul(tf.transpose(X), error) # MSE를 theta에 대해 미분
training_op = tf.assign(theta, theta - learning_rate * gradients) # 반복적인 batch gradient descent step 구현

init = tf.global_variables_initializer() # 훈련 전 변수 초기화

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0: # 100번 반복시마다 MSE의 값 출력
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()    

Epoch 0 MSE = 2.754427
Epoch 100 MSE = 0.63222194
Epoch 200 MSE = 0.5727803
Epoch 300 MSE = 0.5585008
Epoch 400 MSE = 0.54907006
Epoch 500 MSE = 0.542288
Epoch 600 MSE = 0.5373791
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.53124255
Epoch 900 MSE = 0.5293705


In [28]:
best_theta

array([[ 2.06855226e+00],
       [ 7.74078071e-01],
       [ 1.31192386e-01],
       [-1.17845066e-01],
       [ 1.64778143e-01],
       [ 7.44081801e-04],
       [-3.91945131e-02],
       [-8.61356556e-01],
       [-8.23479712e-01]], dtype=float32)

### 자동미분(autodiff) 사용

#### gradients 사용
- 하나의 연산과 변수 리스트를 받아 각 변수에 대한 연산의 gradient를 계산하는 새로운 연산을 만듬.
- tensorflow는 후진모드 자동미분(reverse-mode autodiff)을 사용.

In [29]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

gradients = tf.gradients(mse, [theta])[0] # theta에 대한 MSE의 gradient vector 계산
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print("best_theta:")
print(best_theta)

Epoch 0 MSE = 2.754427
Epoch 100 MSE = 0.63222194
Epoch 200 MSE = 0.5727803
Epoch 300 MSE = 0.5585009
Epoch 400 MSE = 0.54907006
Epoch 500 MSE = 0.542288
Epoch 600 MSE = 0.5373791
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.53124255
Epoch 900 MSE = 0.5293704
best_theta:
[[ 2.06855249e+00]
 [ 7.74078071e-01]
 [ 1.31192386e-01]
 [-1.17845066e-01]
 [ 1.64778143e-01]
 [ 7.44078017e-04]
 [-3.91945094e-02]
 [-8.61356676e-01]
 [-8.23479772e-01]]


#### optimizer 사용

In [30]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse) # theta에 대한 MSE의 gradient vector 계산 

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print("best_theta:")
print(best_theta)

Epoch 0 MSE = 2.754427
Epoch 100 MSE = 0.63222194
Epoch 200 MSE = 0.5727803
Epoch 300 MSE = 0.5585009
Epoch 400 MSE = 0.54907006
Epoch 500 MSE = 0.542288
Epoch 600 MSE = 0.5373791
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.53124255
Epoch 900 MSE = 0.5293704
best_theta:
[[ 2.06855249e+00]
 [ 7.74078071e-01]
 [ 1.31192386e-01]
 [-1.17845066e-01]
 [ 1.64778143e-01]
 [ 7.44078017e-04]
 [-3.91945094e-02]
 [-8.61356676e-01]
 [-8.23479772e-01]]


#### momentum optimizer 사용
- 경사하강법보다 종종 더 빠르게 수렴

In [31]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse) # theta에 대한 MSE의 gradient vector 계산 

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print("best_theta:")
print(best_theta)

Epoch 0 MSE = 2.754427
Epoch 100 MSE = 0.5273161
Epoch 200 MSE = 0.5244147
Epoch 300 MSE = 0.5243281
Epoch 400 MSE = 0.52432185
Epoch 500 MSE = 0.524321
Epoch 600 MSE = 0.52432114
Epoch 700 MSE = 0.52432096
Epoch 800 MSE = 0.52432096
Epoch 900 MSE = 0.52432096
best_theta:
[[ 2.068558  ]
 [ 0.8296167 ]
 [ 0.11875112]
 [-0.26552212]
 [ 0.30569226]
 [-0.00450316]
 [-0.03932616]
 [-0.8998917 ]
 [-0.87054664]]


## 7. 훈련 알고리즘에 데이터 주입

### placeholder node
- 실제 계산을 하지 않는 특수한 노드
- 실행 시 주입한 데이터를 출력한 함

In [24]:
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict = {A : [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict = {A : [[4, 5, 6], [7, 8, 9]]})

In [25]:
print(B_val_1)

[[6. 7. 8.]]


In [26]:
print(B_val_2)

[[ 9. 10. 11.]
 [12. 13. 14.]]


### mini-batch gradient descent algorithm
- 임의의 작은 샘플 세트(mini-batch)에 대해 gradient를 계산

In [ ]:
n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse) 

init = tf.global_variables_initializer()

n_epochs = 10

batch_size = 100
n_batches = int(np.ceil(m / batch_size))

# mini-batch를 추출하기 위한 함수
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  
    indices = np.random.randint(m, size=batch_size)  
    X_batch = scaled_housing_data_plus_bias[indices] 
    y_batch = housing.target.reshape(-1, 1)[indices] 
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

In [ ]:
best_theta

## 8. 모델 저장과 복원